In [1]:
!pip install git+https://github.com/MTG/freesound-python

  Cloning https://github.com/MTG/freesound-python to /tmp/pip-req-build-l9i0o1xp
  Running command git clone --filter=blob:none --quiet https://github.com/MTG/freesound-python /tmp/pip-req-build-l9i0o1xp
  Resolved https://github.com/MTG/freesound-python to commit 5be99a3689d17303c01cb122bbb0d5a96eba04f6
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for freesound-python: filename=freesound_python-1.1-py3-none-any.whl size=6714 sha256=64edda211012bd63e4d0887a7e0ccda9e67ea7e34a6992b56e9f4b7556dfb46e
  Stored in directory: /tmp/pip-ephem-wheel-cache-1fkunnrd/wheels/dc/c8/27/88211ef4ed95f2a550e37a5391838b1351782509677b1545af
Successfully built freesound-python


In [4]:
import requests
import os
import json
import pandas as pd
from tqdm import tqdm
from google.colab import files
from requests_oauthlib import OAuth2Session
from freesound import FreesoundClient
import time

CLIENT_ID = 'xxx'                    ###Enter your freesound 'Client id' from 'Freesound API credentials'###
API_KEY = 'xxx'  ###Enter your freesound 'Client secret/Api key' from 'Freesound API credentials'###

# Set up the Freesound client
client = FreesoundClient()
client.set_token(API_KEY, "token")

# Specify the desired query and fields
query = ["happy"]                                     ###Request a label, for example, happy###
fields = "id,name,previews,tags,analysis"

# Specify the number of sounds to analyze
total_sounds_to_analyze = 400                         ###Request number here###

# Counter for analyzed sounds
analyzed_sounds = 0

# Counter for current page
current_page = 1

# List to store results
results_list = []

# Loop until the desired number of sounds is analyzed
with tqdm(total=total_sounds_to_analyze, desc="Analyzing sounds") as pbar:
    while analyzed_sounds < total_sounds_to_analyze:
        try:
            # Perform a text search for the query with specified fields and page
            results = client.text_search(query=query, fields=fields, page=current_page)

            # Loop through the results and get analysis for each sound
            for sound in results:
                try:
                    # Get the preview link directly from the API
                    mp3_link = sound.previews.preview_lq_mp3 if hasattr(sound.previews, 'preview_lq_mp3') else None

                    # Get the analysis for the sound with retries
                    retry_count = 0
                    max_retries = 3
                    while retry_count < max_retries:
                        try:
                            analysis = sound.get_analysis()
                            break
                        except Exception as e:
                            retry_count += 1
                            time.sleep(3)  # Wait for 3 second before retrying

                    # Convert the analysis to a dictionary
                    analysis_dict = analysis.as_dict() if analysis else None

                    # Include additional sound information
                    sound_info = {
                        "query": query,
                        "id": sound.id,
                        "name": sound.name,
                        "previews": mp3_link,
                        "tags": sound.tags,
                        "analysis": analysis_dict,
                        # Include other fields as needed
                    }

                    # Append the dictionary to the results list
                    results_list.append(sound_info)

                    # Increment the counters
                    analyzed_sounds += 1
                    pbar.update(1)

                    # If the desired number of sounds is analyzed, break out of the loop
                    if analyzed_sounds >= total_sounds_to_analyze:
                        break

                except Exception as e:
                    print(f"Warning: Error analyzing sound with ID {sound.id}. {e}")
                    continue

            # Increment the page counter for the next request
            current_page += 1
            # Introduce a longer delay to avoid hitting the rate limit
            time.sleep(2)

        except Exception as e:
            # Handle the case where no more sounds are found
            if "Rate limit exceeded" in str(e):
                print("Rate limit exceeded. Waiting for 60 seconds before retrying.")
                time.sleep(60)  # Wait for 60 seconds before retrying
                continue
            else:
                print(f"Warning: Only {analyzed_sounds} sounds found. {e}")
                break

# Specify the JSON file name
json_file = f"{query}.json"

# Write the results to the JSON file
with open(json_file, mode="w", encoding="utf-8") as file:
    json.dump(results_list, file, indent=2, default=str)  # Use default=str to handle non-serializable types

# Download the file to the local machine
files.download(json_file)


Analyzing sounds: 100%|██████████| 400/400 [07:11<00:00,  1.08s/it]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>